# ML Model

In [3]:
from covid_data import daily_covid, convert_to_state_date
import mlpipeline as pipeline
# Separate features to use in the model
features_covid = ['municipio_res', 'embarazo', 'edad', 'diabetes', 'epoc', 
                  'asma', 'inmusupr', 'hipertension', 'cardiovascular', 'obesidad', 'tabaquismo', 'muertos']
covid = daily_covid()
data_model = covid[features_covid]

Requesting data to datos abiertos Mexico
Getting zip raw data into directory, will delete soon
Raw data deleted. If you specified filename, clean data will be saved in data directory


In [4]:
print(data_model.shape)
data_model.head()

(225650, 12)


,municipio_res,embarazo,edad,diabetes,epoc,asma,inmusupr,hipertension,cardiovascular,obesidad,tabaquismo,muertos
0,7,0.0,26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,120,0.0,55,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,91,0.0,42,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,20,0.0,55,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,2,0.0,42,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [5]:
# How many missing values do we have?
pipeline.count_nan(data_model)

municipio_res     0
embarazo          0
edad              0
diabetes          0
epoc              0
asma              0
inmusupr          0
hipertension      0
cardiovascular    0
obesidad          0
tabaquismo        0
muertos           0
dtype: int64

In [6]:
#Split training and testing data 
train, test = pipeline.split_data(data_model, 0.2)
print('test obs:', test.shape[0])
print('train obs:', train.shape[0])

test obs: 45130
train obs: 180520
